In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!git clone https://github.com/slj5074/ml-training-camp.git

Cloning into 'ml-training-camp'...
remote: Enumerating objects: 258, done.
remote: Total 258 (delta 0), reused 0 (delta 0), pack-reused 258
Receiving objects: 100% (258/258), 144.33 MiB | 32.02 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [1]:
import numpy as np
import pandas as pd

In [2]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [3]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [4]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [5]:
%%timeit
result_1 = target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 23.4 s per loop


In [17]:
%%timeit
result_2 = target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 281 ms per loop


In [8]:
%load_ext Cython

In [14]:
%%cython --cplus
cimport numpy as cnp
from libcpp.vector cimport vector

cpdef target_mean_v3(data, y_name, x_name):

    cdef dict value_dict = {}
    cdef dict count_dict = {}
    cdef int n = data.shape[0]

    cdef vector[double] result
    result.reserve(n)

    for i in range(n):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1

    for i in range(n):
        result.push_back((value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1))
    return result

In [16]:
%%timeit
result_3 = target_mean_v3(data, 'y', 'x')

1 loop, best of 3: 271 ms per loop
